# ADE Dataset (humans)

> Michal Gallus (s172679) Julien Hoareau (s161088) Wazir Sahebali (s172062)

In this notebook the main evaluation of the network will be demonstrated on the [ADE dataset](http://sceneparsing.csail.mit.edu/) with the purpose of segmenting humans in images. A large part of this network is the same as the [network we applied on the CamVid dataset](./Main%20Notebook.ipynb). The output will be shown in TensorBoard from the `val_ade` folder. In there the image tab will show the photo in the upper left corner, the ground truth in the lower left corner, the difference in the upper right corner (white depicts a wrong prediction), and the predicted segmentation in the lower right corner of each sample.

Below you can specify the amount of test images you want to go through the model. The maximum amount is 532, as there are only that many images.

In [1]:
n_images = 300 # total 532

## Initialization
We import the main libraries along with the 56 version of the network and its parameters. In this application the network only has two classes, i.e. human and non-human.

In [2]:
import os
import numpy as np
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
from network import net
import json
from data_utils_ADE import *
slim = tf.contrib.slim
from tensorflow.python.ops import math_ops, array_ops

classes=2
batch_size, height, width, nchannels = n_images, -1, -1, 3
final_resized = 224
model_version = 56
_mask_labels_ADE = {0: 'nonhuman', 1: 'human'}

with open('model_parameters.json') as params:
    params_dict = json.load(params)[repr(model_version)]
params_dict['input_num_features'] = 48
params_dict['output_classes'] = classes

Below the IoU calculation is defined as we did in the CamVid model, but without any masked weights and for 2 classes only.

In [3]:
def intersection_over_union(ground_truths, predictions, num_classes):
    with tf.variable_scope('iou'):
        batch_size = ground_truths.shape[0]
        iou_array = list()
        for i in range(batch_size):
            gt = tf.reshape(ground_truths[i], [-1])
            pr = tf.reshape(predictions[i], [-1])
            confusion_matrix = tf.confusion_matrix(gt, pr, num_classes, name='cm/' + str(i))
            sum_over_row = math_ops.to_float(math_ops.reduce_sum(confusion_matrix, 0))
            sum_over_col = math_ops.to_float(math_ops.reduce_sum(confusion_matrix, 1))
            cm_diag = math_ops.to_float(array_ops.diag_part(confusion_matrix))
            denominator = sum_over_row + sum_over_col - cm_diag
            # The mean is only computed over classes that appear in the
            # label or prediction tensor. If the denominator is 0, we need to
            # ignore the class.
            num_valid_entries = math_ops.reduce_sum(math_ops.cast(
              math_ops.not_equal(denominator, 0), dtype=tf.float32))
            # If the value of the denominator is 0, set it to 1 to avoid
            # zero division.
            denominator = array_ops.where(
              math_ops.greater(denominator, 0),
              denominator,
              array_ops.ones_like(denominator))
            iou = math_ops.div(cm_diag, denominator)
            iou_array.append(iou)

        iou_array = tf.stack(iou_array)
        return tf.reduce_mean(iou_array, axis=0), tf.reduce_mean(iou_array) 



## Dataloading

The Tensorflow record file (a container containing the dataset), is actually already in the GitHub folder, so there is no need to recreate it. The code for recreating it is similar to the code in the [main notebook](./Main%20Notebook.ipynb#Dataloading). 
If the validation set is not yet present as a TensorFlow Record it will be created here and loaded. Note that if you want to recreate the TensorFlow Record file, you have to change the variable `ADEDIR` to the location where the ADE dataset is stored.

In [4]:
datasetfilename="validationset_ADE.tfrec"
if not os.path.isfile(datasetfilename):
    from convertimages import *
    ADEDIR="../ADEChallengeData2016/" # Change this to where you stored the ADE dataset
    allconvert(ADEDIR) # Sets all non-human classes to 0 and the human class to 1 in all images, all images with humans in it are appended to a list
    tfrec_dump("validation", datasetfilename, ADEDIR)
tfsdataset = slim_dataset(datasetfilename, n_images) 

The type of data transformations applied to the ADE dataset are the same as the ones we applied for the CamVid dataset, i.e. random flipping and random cropping to a resolution of 224 by 224. The one difference is that the ADE dataset contains images of different resolutions, which is why we first resize the smallest side of the image to 224 (maintaining the aspect ratio) and then crop.

In [5]:
def resizecrop_image_and_labels(image, labels, feature_maps_image, feature_maps_annot, height, width):
    """Resizes and randomly crops `image` together with `labels`.
    Based on <https://stackoverflow.com/questions/42147427/tensorflow-how-to-randomly-crop-input-images-and-labels-in-the-same-way>
    """
    seed = random.randint(0, 1e10)
    # We concatenate the image with its annotations as an additional feature map
    combined = tf.concat([image, labels], axis=-1)
    image_shape = tf.shape(image)
    
    # We have to find out which side is smaller
    condition=tf.less(image_shape[0], image_shape[1]) 
    # About the casting: The multiplication has to happen with floats and the eventual resolution has to be in integers
    res_h=tf.cond(condition, lambda: height, lambda: tf.cast(tf.truediv(width,image_shape[1])*tf.cast(image_shape[0], tf.float64), tf.int32))
    res_w=tf.cond(condition, lambda: tf.cast(tf.truediv(height,image_shape[0])*tf.cast(image_shape[1], tf.float64), tf.int32), lambda: width)
    
    # We resize the image
    combined_resize = tf.image.resize_images(
        combined,
        size=[res_h, res_w])
        #Default ResizeMethod.BILINEAR
    last_label_dim = tf.shape(labels)[-1]
    last_image_dim = tf.shape(image)[-1]
    # And then we crop the image
    combined_crop = tf.random_crop(
        combined_resize,
        size=[height, width, feature_maps_image + feature_maps_annot],
        seed=seed)
    combined_crop = tf.reshape(combined_crop, shape=(height, width, feature_maps_image + feature_maps_annot))
    maybe_flipped_images = tf.image.random_flip_left_right(combined_crop)
    crop_feature_maps = tf.unstack(maybe_flipped_images, axis=-1)
    return tf.stack(crop_feature_maps[:feature_maps_image], axis=-1), tf.stack(crop_feature_maps[feature_maps_image:], axis=-1)

## The Network
The network model applied here is exactly the same as in the main notebook. Further explanations of this can thus be found in the [other notebook](./Main%20Notebook.ipynb#The-Network).

Once the data loader is set up, the data can be passed through the network. Below, the weights of the trained network are reloaded and the dataset is passed through the network. The results will be visible in TensorBoard from the `val_ade` folder.

In [6]:
gpu_opts = tf.GPUOptions(per_process_gpu_memory_fraction=0.9)
# Loop
with tf.Session(config=tf.ConfigProto(gpu_options=gpu_opts)) as sess:
    log_dir = 'val_ade'
    # We load a batch and reshape to tensor
    xbatch, ybatch = batch(
        tfsdataset, batch_size=batch_size, height=height, width=width, resized=final_resized)
    input_batch = tf.reshape(xbatch, shape=(batch_size, final_resized, final_resized, 3))
    ground_truth_batch = tf.reshape(ybatch, shape=(batch_size, final_resized, final_resized, 1))

    # Obtain the prediction
    predictions = net(input_batch, params_dict, is_training=False)
   
    # We calculate the loss
    one_hot_labels = slim.one_hot_encoding(
        tf.squeeze(ground_truth_batch),
        params_dict['output_classes'])
    slim.losses.softmax_cross_entropy(
        predictions,
        one_hot_labels)

    # The prediction is softmaxed and the class with the highest probability for each pixel is retained
    predim = tf.nn.softmax(predictions)
    predimmax = tf.expand_dims(tf.cast(tf.argmax(predim, axis=3), tf.float32), -1)
        
    total_loss = slim.losses.get_total_loss()
    tf.summary.scalar('loss', total_loss)
    
    yb = tf.cast(tf.divide(ground_truth_batch, classes), tf.float32)
    
    predim = tf.nn.softmax(predictions)
    predimmax = tf.expand_dims(
        tf.cast(tf.argmax(predim, axis=3), tf.float32), -1)
    predimmaxdiv = tf.divide(tf.cast(predimmax, tf.float32), classes)
    
    # A difference picture is created
    ediff = tf.abs(tf.subtract(yb, predimmaxdiv))
    norm_ediff = tf.ceil(ediff)
    accuracy = tf.reduce_mean(tf.cast(norm_ediff, tf.float32))
    tf.summary.scalar('accuracy', accuracy)
    
    # All 4 images are concatenated into 1
    annots=tf.concat([tf.multiply(input_batch, yb),tf.multiply(input_batch, predimmaxdiv)],2)
    img_and_err=tf.concat([input_batch,tf.image.grayscale_to_rgb(norm_ediff)],2)
    aio=tf.concat([img_and_err,annots],1)
    tf.summary.image("All_in_one", aio, max_outputs=n_images)
    
    # The mean IoU is calculated
    iou_array, mean_iou = intersection_over_union(ground_truth_batch, predimmax, params_dict['output_classes'])
    tf.summary.scalar('mean_IoU', mean_iou)
    class_labels = tf.convert_to_tensor(np.array(list(_mask_labels_ADE.values())), tf.string)
    iou_per_class = tf.stack([class_labels, tf.as_string(iou_array, precision=2)], axis=1)
    tf.summary.text('IoU per class', iou_per_class)

    slim.evaluation.evaluate_once(
        '',
        'train_ade/model.ckpt-46176', # Model weights
        log_dir                       # Save directory for the logs
    ) 